In [ ]:
pip install openai

In [ ]:
pip install requests beautifulsoup4

In [ ]:
pip install validators

In [22]:
import openai
import requests
import json
from bs4 import BeautifulSoup
import validators
import panel as pn # GUI

In [4]:
openai.api_key = 'ENTER OPENAI API HERE'

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0, role="user"):
    messages = [{"role": role, "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    # print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [25]:
url = 'https://about.fb.com/news/2023/09/new-ray-ban-meta-smart-glasses/'

def get_text(url):
  response = requests.get(url) # send an HTTP GET request to the URL

  if response.status_code == 200: # request successful
      soup = BeautifulSoup(response.text, 'html.parser') # parse the HTML content using BeautifulSoup
      article_text = soup.get_text() # extract all the text from the parsed HTML
      # print(article_text)

      return article_text
  else:
      print("Failed to fetch the content. Status code:", response.status_code)


In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

In [ ]:
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are an intelligent website summarizer.\
You greet the user and then ask them to enter the URL of the website in single or double quotes, \
whose content they wish to summarize. Once the URL is provided, the URL will be parsed and \
the HTML text content from the website will be provided to you for summarizing. \
Your job will be to generate and return a concise yet informative summary of the article in \
not more than 10 bullet points. Keep the language formal and follow the logical flow of \
information as in the original article. Also provide a relevant title for your summary. \
You will further offer the following additional options to the user and perform \
the corresponding task they request for: \
1. Translate the summary in a language of their choice. \
2. Paraphrase the summary either in more concise bullets or a paragraph. \
3. Provide further insights and analysis into the topic being discussed in the article, \
such as the tone and sentiment of the article and important themes discussed in the article. \
Keep this short and unbiased, and provide relevant colon-separated labels for what information you are \
providing. For example, format your answer in the following way: Tone: ..., Sentiment:..., \
Themes:... .
4. Provide additional facts, figures and information relevant to the topic being discussed \
in the website article.

"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Enter!", width=100)

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    pn.Row(inp, button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard